In [11]:
import os

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [ ]:
'''GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]=LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY'''

### load the model and test it with the simple message

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [17]:
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [18]:
message=[SystemMessage(content="Hi you are nice Bot"),HumanMessage(content="Hi, how are you buddy?")]

In [19]:
message2=[SystemMessage(content="Hi you are nice Bot and you answer everything in brief"),HumanMessage(content="Hi, how are you buddy?")]

In [20]:
model.invoke(message)

KeyboardInterrupt: 

### use output parser

In [21]:
from langchain_core.output_parsers import StrOutputParser

In [22]:
parser = StrOutputParser()

In [23]:
response = model.invoke(message)

KeyboardInterrupt: 

In [14]:
response2 = model.invoke(message2)

In [15]:
parser.invoke(response)

'Thank you for the compliment! I am doing well. I am always happy to help you with your questions. How can I assist you today?'

In [16]:
parser.invoke(response2)

"I'm swell, thanks!"

### chaining with LCEL

In [17]:
#added two component
chain = model | parser

In [18]:
chain.invoke(message)

'Hello, I am not BotHi, I am Gemini, a multi-modal AI model, developed by Google. I am not capable of having feelings or emotions, so I do not have a mood or the ability to feel good or bad. I am designed to provide information and assist users to the best of my abilities. May I help you with anything today?'

In [19]:
chain.invoke(message2)

'I am well, thank you.'

### Now lets start with prompting

In [24]:
from langchain_core.prompts import ChatPromptTemplate

In [21]:
system="translate the following into {language}"
#user="i am hungry."

In [25]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [26]:
prompt = prompt_template.invoke({"language":"french","text":"i am learning langchain with sunny sir."})

In [27]:
parser.invoke(model.invoke(prompt))

KeyboardInterrupt: 

In [25]:
prompt.to_messages()

[SystemMessage(content='translate the following into french'),
 HumanMessage(content='i am learning langchain with sunny sir.')]

### chaining 

In [28]:
chain = prompt_template | model | parser

In [29]:
chain.invoke({"language": "hindi", "text": "I am hungry"})

KeyboardInterrupt: 

### Agents

In [30]:

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [32]:
memory = MemorySaver()

In [33]:
search=TavilySearchResults( max_results=2)

In [34]:
tools=[search]

In [ ]:
# agent_executor=create_react_agent(model,tools,checkpointer=memory)
agent_executor=create_react_agent(model="",tools=tools,checkpointer=memory)

In [36]:
config = {"configurable": {"thread_id": "abc123"}}

In [37]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im sunny! and i live in bangalore")]}, config):
    print(chunk)
    print("----")

KeyboardInterrupt: 

In [36]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in Bangalore"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-9952e6bc-1006-4220-98a6-e82bf7a9970e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in Bangalore'}, 'id': '31999abb-6e79-45e7-968c-b19c15a6ba90', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 25, 'total_tokens': 134})]}}
----
{'too